# Fixing MISSING DWI

In this section we verify that the subjects with the `DWI_MISSING_BVAL` errors are in fact missing data. Because these subjects were copied from PNC we will verify in that project that there are no `BVAL`s and `BVEC`s available

In [1]:
import flywheel
import pandas as pd
from fw_heudiconv.cli.export import get_nested

client = flywheel.Client()

/home/ttapera/miniconda3/envs/flywheel/lib/python3.7/site-packages/fw_heudiconv/backend_funcs/query.py:4: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  from nibabel.nicom.dicomwrappers import wrapper_from_data


200910-18:52:13,328 nipype.utils INFO:
	 Running nipype version 1.4.0 (latest: 1.5.0)


INFO: Running nipype version 1.4.0 (latest: 1.5.0)


In [2]:
df = pd.read_csv('/storage/ttapera/RBC/PennLINC/Validation/PNC/validation.csv')
echotimes_rows = df.loc[df['code'] == 33,]

In [3]:
echotimes_rows.subject

5036    sub-3392035756
Name: subject, dtype: object

We're going to confirm that these cases are all problematic because they have no fieldmaps to begin with in the PNC project:

In [4]:
# get the subject ids <-> bblids

rbc_subjects = list(echotimes_rows.subject)
df2 = pd.read_csv('/storage/ttapera/RBC/data/bblids_3.csv', names=['bblid', 'hash', 'fullid', 'process'])

pnc_subjects = df2[df2['fullid'].isin(rbc_subjects)]
pnc_subjects = [str(i) for i in list(pnc_subjects.bblid)]

In [5]:
# do some flywheel stuff

pnc_proj = client.projects.find_first('label=PNC_CS_810336')
subjects = pnc_proj.subjects()
subjects2 = [sub for sub in subjects if sub.label in pnc_subjects]

In [6]:
subjects_dict = dict(zip(pnc_subjects, rbc_subjects))

In [9]:
subject = subjects2[0]

session = subject.sessions()[0]

acquisitions = session.acquisitions()

for acq in acquisitions:
    acq = client.get(acq.id)
    print(acq.label)
    print("Available Files")
    print([x.name for x in acq.files])
    print()

9
Available Files
['STIM_catalog.xml', 'ASSOCIATED_catalog.xml', '9.dicom.zip', 'DTI_2x32_36_9.nii.gz']

8
Available Files
['ASSOCIATED_catalog.xml', 'STIM_catalog.xml', '8.dicom.zip', 'DTI_2x32_35_8.nii.gz', 'DTI_2x32_35_8.bvec', 'DTI_2x32_35_8.bval']

7
Available Files
['STIM_catalog.xml', 'frac2back', 'ASSOCIATED_catalog.xml', '7.dicom.zip', 'bbl1_frac2back1_231_7.nii.gz']

6
Available Files
['STIM_catalog.xml', 'idemo', 'ASSOCIATED_catalog.xml', '6.dicom.zip', 'bbl1_idemo2_210_6.nii.gz']

5
Available Files
['STIM_catalog.xml', 'ASSOCIATED_catalog.xml', '5.dicom.zip', 'epi_singlerep_advshim_5.nii.gz']

41
Available Files
['PRESENTATION_catalog.xml', '02923-frac2b_1.00.log']

4
Available Files
['STIM_catalog.xml', 'ASSOCIATED_catalog.xml', '4.dicom.zip', 'ep2d_se_pcasl_PHC_1200ms_4.nii.gz']

37
Available Files
['02923-idemo2.10.log', 'PRESENTATION_catalog.xml']

3
Available Files
['STIM_catalog.xml', 'ASSOCIATED_catalog.xml', '3.dicom.zip', 'ep2d_se_pcasl_PHC_1200ms_3.nii.gz']

2
Ava

Acquisition 9 is the offensive acquisition. We can see that there is a DWI but there is no accompanying BVAL & BVEC. Running `dcm2niix v1.0.20190902` on the file outputs only the NIfTI file.